In [14]:
## Fetching the CensusTract Details in the Business Data using GEO PANDAS

In [1]:
##Install GEO-PANADS
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [15]:
business = r"/Users/kavyavempati/Desktop/Kavya/Capstone/Legally_Operating_Businesses_20231020.csv"##Business Files 
census = r"/Users/kavyavempati/Desktop/Kavya/Capstone/cb_2022_36_tract_500k"   ##Census Tract Shape Files
taxi_zone= r"/Users/kavyavempati/Desktop/Kavya/Capstone/taxi_zones- shape files" ## Taxi Zone Shape Files

excel_output1 =r"/Users/kavyavempati/Desktop/Kavya/Capstone/census_wZones.xlsx"  ## Census Vs Zones
excel_output2 = r"/Users/kavyavempati/Desktop/Kavya/Capstone/zones_wCensus.xlsx"  ## Zones Vs Census
excel_output3  = r"/Users/kavyavempati/Desktop/Kavya/business_zones_wCensus.xlsx"  #Businesss Data

In [16]:
###READING BUSINESS DATA
busines_df = pd.read_csv(business) 
pts= busines_df.apply (lambda row: Point (row. Longitude, row. Latitude), axis=1) 
gdf = gpd. GeoDataFrame (busines_df, geometry=pts,crs="EPSG:4326") 
##gdf.crs = "EPSG: 4326"

/var/folders/31/5ccmk1h91x1fbrmmcd39jt5m0000gp/T/ipykernel_34787/3855573947.py:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  busines_df = pd.read_csv(business)


In [17]:
##Create Dataframe for Census file
census_df=gpd.read_file(census) 

In [18]:
##Create Dataframe for Taxi Zones file
taxi_zones_df = gpd.read_file(taxi_zone)

In [19]:
print(census_df.crs, taxi_zones_df.crs)

GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]] PROJCS["NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",40.6666666666667],PARAMETER["standard_parallel_2",41.0333333333333],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["Foot_US",0.304800609601219],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [20]:
#Converting to same as zone projection to align them
census_df = census_df.to_crs(taxi_zones_df.crs)
business_gdf = gdf.to_crs(taxi_zones_df.crs)

In [21]:
print(census_df.crs, taxi_zones_df.crs)

PROJCS["NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",40.6666666666667],PARAMETER["standard_parallel_2",41.0333333333333],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["Foot_US",0.304800609601219],AXIS["Easting",EAST],AXIS["Northing",NORTH]] PROJCS["NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",40.6666666666667]

In [22]:
#Same as taxi zone , making census data as well same as zone
clipped_census = gpd.clip(census_df, taxi_zones_df)

In [23]:
#Converting multi parts to single parts to get seperate centriods
clipped_census_MS = clipped_census.explode(ignore_index=True)

In [24]:
#Business Pts with Census - Many to One
business_gdf_sj = gpd.sjoin(business_gdf,clipped_census_MS,how='left')

In [25]:
#Business Pts with Census with Zones - Many to One
business_gdf_sj = business_gdf_sj.rename(columns={'index_right':'old_index_right'})
business_gdf_sj_2 = gpd.sjoin(business_gdf_sj,taxi_zones_df,how='left')

In [26]:
business_gdf_sj_2 .to_excel(excel_output3, index=False)

In [13]:
business_gdf_sj_2.to_file(r"/Users/kavyavempati/Desktop/Kavya/Capstone/BU_SP.shp")

/var/folders/31/5ccmk1h91x1fbrmmcd39jt5m0000gp/T/ipykernel_34787/2033273497.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  business_gdf_sj_2.to_file(r"/Users/kavyavempati/Desktop/Kavya/Capstone/BU_SP.shp")
